In [ ]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib.pyplot as plt
# ceļu norādīt nevajag, ja pakotne ir instalēta
documentpath=os.getcwd()
libpath=os.path.abspath(os.path.join(documentpath, os.pardir))
sys.path.append(libpath)
from sudrabainiemakoni.cloudimage import CloudImage
from sudrabainiemakoni.cloudimage import WebMercatorImage
from sudrabainiemakoni import plots

In [ ]:
# katalogs ar piemēra failiem
sample_directory = 'SampleData'
# katalogs ar rezultātu failiem
results_directory = 'SampleResults'
case_id = 'js_202106210100'

In [ ]:
# ielasām projektu
# piemēra projekta failu var iegūt ar ReferencetKameru izklājloksni
cldim = CloudImage.load(f'{results_directory}/{case_id}.proj')

### Projicētā attēla iegūšana

In [ ]:
# projicēšanas apgabals un izšķirtspēja
lonmin, lonmax, latmin, latmax, horizontal_resolution_km = 15, 35, 57, 64, 0.5

In [ ]:
# sagatavojam projicēšanas objektu
webmerc = WebMercatorImage(cldim, lonmin, lonmax, latmin, latmax, horizontal_resolution_km)

Sagatavojam projicēšanu uz noteiktu augstumu - var projicēt arī uz webmerc objekta izšķirtspējai atbilstošo augstumu masīvu. Šādu masīvu var iegūt izgatavojot to no divu attēlu referencēšanas (sk objektu *CloudImagePair*)

In [ ]:
height = 80 # kilometros
webmerc.prepare_reproject_from_camera(height)

Iegūstam projicēto attēlu masīvā  
Tas ir attēls ar masku alpha kanālā, kas maskē telpas apgabalu ārpus oriģinālā attēla  
Ja ir nepieciešams pie vienas un tās pašas augstumu kartes projicēt vairākus dažādus attēlus (piem animācijai), tad pietiek vienreiz izsaukt prepare_reproject_from_camera un tad mainīt cldim attēlu (cldim.filename=  cldim.LoadImage(reload=True))

In [ ]:
projected_image=webmerc.Fill_projectedImageMasked()

In [ ]:
plt.imshow(projected_image)
#projected_image.shape

Noglabājam referencētu tiff vai jpg lietošanai GIS programmās (piem. QGIS)  
Referencēšanu noglabājam tfw vai jgw tipa teksta failos  
Attēls būs WebMercator projekcijā ([EPSG:3857](https://epsg.io/3857)) 
Ielasot to QGIS ar Layer->Add Layer->Add raster layer, jānorāda atbilstošā koordinātu sistēma.

In [ ]:
import skimage.io
fntiff=f"{results_directory}/proj_{height}_{case_id}.tif"
skimage.io.imsave(fntiff, projected_image)
webmerc.SaveJgw(os.path.splitext(fntiff)[0]+'.tfw')

### Projicētā attēla attēlošana uz OpenStreetMap pamatnes

In [ ]:
map_lonmin, map_lonmax, map_latmin, map_latmax = 15,33,56,64
pp=[[cldim.location.lon.value, cldim.location.lat.value]]
plots.PlotReferencedImages(webmerc, [projected_image],  camera_points=pp,
                               outputFileName=None,
                               lonmin=map_lonmin, lonmax=map_lonmax, latmin=map_latmin, latmax=map_latmax,
                               showplot=True,
                               alpha=0.8)
